In [265]:
# !pip install geopy #Install it once and then comment it

In [266]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
pd.set_option("display.max_colwidth",None)
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)

from IPython.display import display, HTML
display(HTML("<h1>Airbnb Data</h1>"))
from geopy.distance import great_circle

In [267]:
crime_data = pd.read_csv("Crime_Data_from_2020_to_Present.csv")
airbnb_data = pd.read_csv("listings.csv")

In [268]:
print(airbnb_data.shape)
print(crime_data.shape)

(44684, 18)
(351325, 28)


In [269]:
crime_data.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,211310365,05/28/2021 12:00:00 AM,05/28/2021 12:00:00 AM,1250,13,Newton,1361,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",1300 0400 2024 0329 1307,23,F,H,122.0,"VEHICLE, PASSENGER/TRUCK",310.0,CONCRETE BLOCK/BRICK,IC,Invest Cont,740.0,NaN,NaN,NaN,300 W 49TH ST,NaN,33.9983,-118.2802
1,210207810,04/06/2021 12:00:00 AM,04/06/2021 12:00:00 AM,1850,2,Rampart,247,2,626,INTIMATE PARTNER - SIMPLE ASSAULT,2000 0913 1814 0448 0408 0400 1310 1309,27,F,B,122.0,"VEHICLE, PASSENGER/TRUCK",400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,626.0,NaN,NaN,NaN,5TH ST,UNION AV,34.0596,-118.2701
2,211806706,02/24/2021 12:00:00 AM,02/23/2021 12:00:00 AM,2100,18,Southeast,1802,1,510,VEHICLE - STOLEN,NaN,0,NaN,NaN,101.0,STREET,NaN,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,200 W 88TH PL,NaN,33.9564,-118.2756
3,211008058,05/01/2021 12:00:00 AM,04/29/2021 12:00:00 AM,1620,10,West Valley,1075,1,330,BURGLARY FROM VEHICLE,1300 0358 1202 1822 1609 0344,63,F,W,719.0,MEDICAL/DENTAL OFFICES,NaN,NaN,IC,Invest Cont,330.0,NaN,NaN,NaN,VENTURA BL,LINDLEY AV,34.1654,-118.5272
4,210817701,12/05/2021 12:00:00 AM,07/09/2021 12:00:00 AM,1040,8,West LA,899,2,354,THEFT OF IDENTITY,0100 0928 1822,33,M,Z,501.0,SINGLE FAMILY DWELLING,NaN,NaN,IC,Invest Cont,354.0,NaN,NaN,NaN,3200 S SHERBOURNE DR,NaN,34.0319,-118.3829


In [270]:
airbnb_data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,109,Amazing bright elegant condo park front *UPGRADED*,521,Paolo,Other Cities,Culver City,33.98301,-118.38607,Entire home/apt,129.0,30,2,2016-05-15,0.01,1,13,0,NaN
1,2708,Runyon Canyon | Beau Furn Mirror Mini-Suite Firplc,3008,Chas.,City of Los Angeles,Hollywood,34.09625,-118.34605,Private room,89.0,30,41,2024-03-07,0.34,2,317,2,NaN
2,24349,"The Wolford House, a Mid Century Hilltop Jewel",98565,Zara,City of Los Angeles,Mount Washington,34.10942,-118.22003,Entire home/apt,450.0,30,175,2024-03-20,1.62,1,296,8,HSR21-002783
3,2732,Zen Life at the Beach,3041,Yoga Priestess,Other Cities,Santa Monica,34.00440,-118.48095,Private room,139.0,7,24,2022-08-21,0.15,2,294,0,228269
4,2864,* Beautiful Master Suite/Jacuzzi Tub/*,3207,Bernadine,Other Cities,Bellflower,33.87490,-118.11515,Entire home/apt,NaN,30,0,NaN,NaN,1,0,0,NaN


In [271]:
crime_data.isnull().sum()

DR_NO                  0
Date Rptd              0
DATE OCC               0
TIME OCC               0
AREA                   0
AREA NAME              0
Rpt Dist No            0
Part 1-2               0
Crm Cd                 0
Crm Cd Desc            0
Mocodes            47931
Vict Age               0
Vict Sex           45493
Vict Descent       45494
Premis Cd              4
Premis Desc          282
Weapon Used Cd    233961
Weapon Desc       233961
Status                 0
Status Desc            0
Crm Cd 1               6
Crm Cd 2          327565
Crm Cd 3          350562
Crm Cd 4          351304
LOCATION               0
Cross Street      300250
LAT                    0
LON                    0
dtype: int64

In [272]:
airbnb_data.isnull().sum()

id                                    0
name                                  1
host_id                               0
host_name                             2
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                              8144
minimum_nights                        0
number_of_reviews                     0
last_review                       12114
reviews_per_month                 12114
calculated_host_listings_count        0
availability_365                      0
number_of_reviews_ltm                 0
license                           31896
dtype: int64

In [273]:
crime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351325 entries, 0 to 351324
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DR_NO           351325 non-null  int64  
 1   Date Rptd       351325 non-null  object 
 2   DATE OCC        351325 non-null  object 
 3   TIME OCC        351325 non-null  int64  
 4   AREA            351325 non-null  int64  
 5   AREA NAME       351325 non-null  object 
 6   Rpt Dist No     351325 non-null  int64  
 7   Part 1-2        351325 non-null  int64  
 8   Crm Cd          351325 non-null  int64  
 9   Crm Cd Desc     351325 non-null  object 
 10  Mocodes         303394 non-null  object 
 11  Vict Age        351325 non-null  int64  
 12  Vict Sex        305832 non-null  object 
 13  Vict Descent    305831 non-null  object 
 14  Premis Cd       351321 non-null  float64
 15  Premis Desc     351043 non-null  object 
 16  Weapon Used Cd  117364 non-null  float64
 17  Weapon Des

In [274]:
airbnb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44684 entries, 0 to 44683
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              44684 non-null  int64  
 1   name                            44683 non-null  object 
 2   host_id                         44684 non-null  int64  
 3   host_name                       44682 non-null  object 
 4   neighbourhood_group             44684 non-null  object 
 5   neighbourhood                   44684 non-null  object 
 6   latitude                        44684 non-null  float64
 7   longitude                       44684 non-null  float64
 8   room_type                       44684 non-null  object 
 9   price                           36540 non-null  float64
 10  minimum_nights                  44684 non-null  int64  
 11  number_of_reviews               44684 non-null  int64  
 12  last_review                     

In [275]:
crime_data.describe()

,DR_NO,TIME OCC,AREA,Rpt Dist No,Part 1-2,Crm Cd,Vict Age,Premis Cd,Weapon Used Cd,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LAT,LON
count,3.513250e+05,351325.000000,351325.000000,351325.000000,351325.000000,351325.000000,351325.000000,351321.000000,117364.000000,351319.000000,23760.000000,763.000000,21.000000,351325.000000,351325.000000
mean,2.244355e+08,1319.144769,10.623383,1108.960091,1.416890,493.983554,29.728465,314.716558,363.116245,493.742354,962.171675,986.647444,993.095238,34.072423,-118.348281
std,4.929497e+06,657.240818,6.140776,613.966029,0.493045,205.479526,21.751326,218.997786,124.321437,205.270811,104.462940,43.652321,15.658559,0.241448,0.754360
min,2.203000e+03,1.000000,1.000000,101.000000,1.000000,110.000000,-1.000000,101.000000,101.000000,110.000000,210.000000,626.000000,946.000000,0.000000,-118.667600
25%,2.208095e+08,846.000000,5.000000,566.000000,1.000000,331.000000,7.000000,102.000000,311.000000,331.000000,998.000000,998.000000,998.000000,34.016300,-118.428700
50%,2.216087e+08,1400.000000,11.000000,1133.000000,1.000000,440.000000,31.000000,210.000000,400.000000,440.000000,998.000000,998.000000,998.000000,34.058700,-118.319700
75%,2.306072e+08,1845.000000,16.000000,1612.000000,2.000000,626.000000,45.000000,501.000000,400.000000,626.000000,998.000000,998.000000,998.000000,34.162900,-118.273900
max,2.399165e+08,2359.000000,21.000000,2198.000000,2.000000,956.000000,99.000000,974.000000,516.000000,956.000000,999.000000,999.000000,999.000000,34.334300,0.000000


In [276]:
airbnb_data.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
count,4.468400e+04,4.468400e+04,44684.000000,44684.000000,36540.000000,44684.000000,44684.000000,32570.000000,44684.000000,44684.000000,44684.000000
mean,4.758161e+17,1.879065e+08,34.054948,-118.312181,317.199781,17.798563,35.987826,1.431085,19.849476,186.152247,8.458822
std,4.740239e+17,1.840032e+08,0.141324,0.172939,1103.192069,28.593314,79.494853,1.729366,78.531417,130.266401,16.584413
min,1.090000e+02,5.210000e+02,33.338360,-118.917134,6.000000,1.000000,0.000000,0.010000,1.000000,0.000000,0.000000
25%,3.182939e+07,2.555165e+07,33.998074,-118.413557,99.000000,2.000000,0.000000,0.210000,1.000000,66.000000,0.000000
50%,5.861918e+17,1.112194e+08,34.061044,-118.343584,160.000000,14.000000,5.000000,0.750000,2.000000,183.000000,1.000000
75%,9.479489e+17,3.487454e+08,34.108332,-118.231011,276.000000,30.000000,33.000000,2.160000,9.000000,310.000000,8.000000
max,1.172845e+18,5.817802e+08,34.811183,-117.654300,99999.000000,1124.000000,2903.000000,56.330000,644.000000,365.000000,613.000000


In [277]:
airbnb_data["host_id"].nunique()

22941

In [278]:
airbnb_data["host_name"].nunique()

8523

In [279]:
airbnb_data = airbnb_data.drop('name',axis=1)

In [280]:
for i in airbnb_data["neighbourhood_group"].unique():
    print(i) 

Other Cities
City of Los Angeles
Unincorporated Areas


In [281]:
airbnb_data = airbnb_data.drop('neighbourhood_group',axis=1)

In [282]:
airbnb_data["host_name"].unique()

array(['Paolo', 'Chas.', 'Zara', ..., 'Zoltan', 'Jeanna Paola', 'Muaz'],
      dtype=object)

In [283]:
airbnb_data = airbnb_data.drop('host_name',axis=1)
airbnb_data = airbnb_data.drop('host_id',axis=1)
airbnb_data = airbnb_data.drop('minimum_nights',axis=1)

In [284]:
for i in airbnb_data["license"].unique():
    print(i) 

nan
HSR21-002783
228269
Exempt - This listing is a bed and breakfast
HSR19-003514
HSR21-001474
Exempt - This listing is a transient occupancy residential structure
HSR19-003021
HSR19-000600
HSR19-000906
HSR21-002074
HSR23-003866
HSR19-001225
HSR24-001527
HSR19-001336
HSR19-001190
HSR23-003959
Exempt
HSR23-002321
HSR21-000113
HSR21-001664
HSR19-003164
Exempt - This listing is a hotel or motel
HSR19-003866
HSR19-002716
HSR19-003488
HSR21-001488
HSR19-003197
HSR19-004121
HSR19-002064
HSR23-000312
HSR19-003150
HSR22-003839
HSR19-004125
HSR23-003916
HSR19-002506
HSR19-003172
HSR21-003398
HSR23-001393
HSR24-000129
HSR19-003988
HSR20-002235
SRU2018-00006
HSR19-002347
HSR21-001354
HSR20-001333
HSR19-003035
HSR19-002570
HSR19-003102
STR20-0017
HSR19-000804
HSR21-003167
HSR19-004793
HSR19-003546
HSR20-000751
222035
HSR23-001821
HSR19-002721
43181
HSR21-001039
PRP22-00864
241180
HSR20-001460
SRH-2018-00018
HSR19-000417
HSR23-000331
STR19-0013
HSR19-003219
HSR19-004128
HSR22-003513
HSR19-003921
HS

In [285]:
airbnb_data = airbnb_data.drop('license',axis=1)
airbnb_data = airbnb_data.drop('number_of_reviews',axis=1)
airbnb_data = airbnb_data.drop('last_review',axis=1)
airbnb_data = airbnb_data.drop('reviews_per_month',axis=1)
airbnb_data = airbnb_data.drop('availability_365',axis=1)
airbnb_data = airbnb_data.drop('number_of_reviews_ltm',axis=1)

In [286]:
airbnb_data["room_type"].unique()

array(['Entire home/apt', 'Private room', 'Shared room', 'Hotel room'],
      dtype=object)

In [287]:
airbnb_data = airbnb_data.drop('room_type',axis=1)

# Crime Data

In [288]:
crime_data = crime_data.drop('DR_NO',axis=1)
crime_data = crime_data.drop('DATE OCC',axis=1)
crime_data = crime_data.drop('Date Rptd',axis=1)
crime_data = crime_data.drop('AREA',axis=1)
crime_data = crime_data.drop('Rpt Dist No',axis=1)
crime_data = crime_data.drop('Vict Age',axis=1)
crime_data = crime_data.drop('Vict Sex',axis=1)
crime_data = crime_data.drop('Vict Descent',axis=1)
crime_data = crime_data.drop('Status',axis=1)
crime_data = crime_data.drop('Status Desc',axis=1)
crime_data = crime_data.drop('Crm Cd 2',axis=1)
crime_data = crime_data.drop('Crm Cd 3',axis=1)
crime_data = crime_data.drop('Crm Cd 4',axis=1)
crime_data = crime_data.drop('Cross Street',axis=1)
crime_data = crime_data.drop('Weapon Used Cd',axis=1)
crime_data = crime_data.drop('Weapon Desc',axis=1)
crime_data = crime_data.drop('Premis Cd',axis=1)
crime_data = crime_data.drop('TIME OCC',axis=1)
crime_data["LAT"].dropna(inplace=True)
crime_data["LON"].dropna(inplace=True)
airbnb_data["latitude"].dropna(inplace=True)
airbnb_data["longitude"].dropna(inplace=True)

In [289]:
crime_data = crime_data.iloc[:-1,:]
airbnb_data = airbnb_data.iloc[:-1,:]

In [290]:
print(crime_data["LON"].isnull().sum())
print(crime_data["LAT"].isnull().sum())
print(airbnb_data["latitude"].isnull().sum())
print(airbnb_data["longitude"].isnull().sum())

0
0
0
0


In [291]:
airbnb_data.head()

,id,neighbourhood,latitude,longitude,price,calculated_host_listings_count
0,109,Culver City,33.98301,-118.38607,129.0,1
1,2708,Hollywood,34.09625,-118.34605,89.0,2
2,24349,Mount Washington,34.10942,-118.22003,450.0,1
3,2732,Santa Monica,34.00440,-118.48095,139.0,2
4,2864,Bellflower,33.87490,-118.11515,NaN,1


In [292]:
crime_data.head()

,AREA NAME,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Premis Desc,Crm Cd 1,LOCATION,LAT,LON
0,Newton,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)",1300 0400 2024 0329 1307,"VEHICLE, PASSENGER/TRUCK",740.0,300 W 49TH ST,33.9983,-118.2802
1,Rampart,2,626,INTIMATE PARTNER - SIMPLE ASSAULT,2000 0913 1814 0448 0408 0400 1310 1309,"VEHICLE, PASSENGER/TRUCK",626.0,5TH ST,34.0596,-118.2701
2,Southeast,1,510,VEHICLE - STOLEN,NaN,STREET,510.0,200 W 88TH PL,33.9564,-118.2756
3,West Valley,1,330,BURGLARY FROM VEHICLE,1300 0358 1202 1822 1609 0344,MEDICAL/DENTAL OFFICES,330.0,VENTURA BL,34.1654,-118.5272
4,West LA,2,354,THEFT OF IDENTITY,0100 0928 1822,SINGLE FAMILY DWELLING,354.0,3200 S SHERBOURNE DR,34.0319,-118.3829


In [293]:
print(airbnb_data.shape)
print(crime_data.shape)

(44683, 6)
(351324, 10)


In [294]:
airbnb_data.isnull().sum()

id                                   0
neighbourhood                        0
latitude                             0
longitude                            0
price                             8144
calculated_host_listings_count       0
dtype: int64

In [295]:
crime_data.isnull().sum()

AREA NAME          0
Part 1-2           0
Crm Cd             0
Crm Cd Desc        0
Mocodes        47931
Premis Desc      282
Crm Cd 1           6
LOCATION           0
LAT                0
LON                0
dtype: int64

In [296]:
# pip install swifter tqdm

In [297]:
from geopy.distance import great_circle
from joblib import Parallel, delayed
import pandas as pd
import os
import time

# Function to process each row
def process_row(row, crime_data):
    nearestRow = None
    minDistance = float('inf')
    for _, crimeRow in crime_data.iterrows():
        # Calculate the distance between the Airbnb location and the crime location
        distance = great_circle((row["latitude"], row["longitude"]),
                                (crimeRow["LAT"], crimeRow["LON"])).meters
        if distance < minDistance:
            minDistance = distance
            nearestRow = crimeRow
    return pd.Series(nearestRow)

# Check the number of rows already processed in the CSV file
def get_processed_rows(output_file):
    if os.path.exists(output_file):
        return len(pd.read_csv(output_file))
    return 0

# Parallel processing function for a batch of data with resuming and logging
def process_in_batches(data, crime_data, batch_size=2000, output_file="MergedData.csv"):
    num_batches = (len(data) // batch_size) + 1
    total_rows = len(data)

    # Get the number of rows already processed
    processed_rows = get_processed_rows(output_file)

    print(f"\nTotal rows in dataset: {total_rows}")
    print(f"Rows already processed: {processed_rows}")

    total_start_time = time.time()

    for batch_index in range(processed_rows // batch_size, num_batches):
        start_row = batch_index * batch_size
        end_row = start_row + batch_size
        batch_data = data.iloc[start_row:end_row]

        if not batch_data.empty:
            batch_start_time = time.time()
            print(f"\n[Batch {batch_index + 1}/{num_batches}] Starting processing (Rows {start_row + 1} to {end_row})")

            # Use Parallel to process each row across all CPU cores
            batch_result = Parallel(n_jobs=-1)(delayed(process_row)(row, crime_data) for _, row in batch_data.iterrows())
            batch_result = pd.DataFrame(batch_result)

            # Append the result to the CSV
            batch_result.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)

            batch_end_time = time.time()
            batch_time_elapsed = batch_end_time - batch_start_time
            total_rows_done = end_row
            total_rows_left = total_rows - total_rows_done
            percent_complete = (total_rows_done / total_rows) * 100
            total_time_elapsed = time.time() - total_start_time

            # Logging detailed progress
            print(f"Batch {batch_index + 1} completed in {batch_time_elapsed:.2f} seconds.")
            print(f"Total rows processed: {total_rows_done}/{total_rows}")
            print(f"Rows left: {total_rows_left}")
            print(f"Progress: {percent_complete:.2f}% completed.")
            print(f"Total time elapsed: {total_time_elapsed // 60:.0f} minutes and {total_time_elapsed % 60:.0f} seconds.")

    total_time_elapsed = time.time() - total_start_time
    print(f"\nAll batches completed. Total time: {total_time_elapsed // 60:.0f} minutes and {total_time_elapsed % 60:.0f} seconds.")

# Example usage
batch_size = 2000
# process_in_batches(airbnb_data, crime_data, batch_size=batch_size, output_file="MergedData.csv")


Total rows in dataset: 44683
Rows already processed: 2000

[Batch 2/23] Starting processing (Rows 2001 to 4000)
Batch 2 completed in 1794.07 seconds.
Total rows processed: 4000/44683
Rows left: 40683
Progress: 8.95% completed.
Total time elapsed: 29 minutes and 54 seconds.

[Batch 3/23] Starting processing (Rows 4001 to 6000)
Batch 3 completed in 1723.00 seconds.
Total rows processed: 6000/44683
Rows left: 38683
Progress: 13.43% completed.
Total time elapsed: 58 minutes and 37 seconds.

[Batch 4/23] Starting processing (Rows 6001 to 8000)
Batch 4 completed in 1738.00 seconds.
Total rows processed: 8000/44683
Rows left: 36683
Progress: 17.90% completed.
Total time elapsed: 87 minutes and 35 seconds.

[Batch 5/23] Starting processing (Rows 8001 to 10000)
Batch 5 completed in 1720.98 seconds.
Total rows processed: 10000/44683
Rows left: 34683
Progress: 22.38% completed.
Total time elapsed: 116 minutes and 16 seconds.

[Batch 6/23] Starting processing (Rows 10001 to 12000)
Batch 6 complet

In [298]:
# crime_data["Crm Cd"].unique()

In [299]:
# crime_data["Crm Cd Desc"].unique()

In [300]:
# crime_data["AREA NAME"].unique()